In [6]:
%config InlineBackend.rc = {"figure.dpi": 72, 'figure.figsize': (6.0, 4.0)}
%matplotlib ipympl

import ase 
import matplotlib.pyplot as plt
import numpy as np
import abtem

abtem.config.set({"local_diagnostics.progress_bar": True})
abtem.config.set({"device": "cpu"});

In [15]:
atoms = ase.build.mx2(vacuum=3)
atoms = abtem.orthogonalize_cell(atoms) * (3, 2, 1)

probe = abtem.Probe(
    semiangle_cutoff=np.linspace(20, 30, 3),
    defocus=np.linspace(0, 50, 3),
    Cs=np.linspace(0, 5e5, 3),
    #astigmatism_angle=np.pi / 4,
    sampling=0.08,
    energy=80e3,
)

potential = abtem.Potential(atoms, slice_thickness=2)

scan = abtem.GridScan(
    start=(0, 0),
    end=(1 / 3, 1 / 2),
    fractional=True,
    potential=potential,
)

doses = np.geomspace(1e1, 1e7, 7)

In [18]:
detectors = abtem.FlexibleAnnularDetector(step_size=5)

In [27]:
measurements = (
    probe.scan(potential, scan=scan)
    .to_image_ensemble()
    .gaussian_filter(0.3)
    .interpolate(0.1)
    .tile((3, 2))
    .no_base_chunks()
    .poisson_noise(np.linspace(10, 1e5, 6))
)

In [28]:
measurements.compute()

[########################################] | 100% Completed | 13.12 s


In [29]:
visualization = measurements.ensure_lazy().show(interact=True, cbar=True)